# IESB - Miner II - Aula 03 - Random Forest

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# **Importação e visualização dos dados**

In [ ]:
# Importando os arquivos
df = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

df.shape, test.shape

In [ ]:
# Verificando o dataframe de treino
df.info()

In [ ]:
# Verificando o dataframe de teste
test.info()

In [ ]:
# Vamos verificar os dados de treino
df.head(10)

In [ ]:
# Vamos verificar os dados de test
test.head(10)

# **Realizando as transformações nos dados**

In [ ]:
# Verificando a distribuição da variável count
df['count'].plot.hist(bins=50)

In [ ]:
# Valor máximo de count
df['count'].max()

In [ ]:
# Que dia e hora teve mais bicicletas alugadas
df[df['count'] == df['count'].max()]

In [ ]:
# Usando nlargest - 5 dias com maior número de aluguel de biciletas
df.nlargest(5, 'count')

In [ ]:
# Todas as bicicletas que foram alugadas por hora no dia que teve a maior quantidade de bicicletas alugadas
df[df['datetime'].str.contains('2012-09-12')]

In [ ]:
# Valor mínimo de count
df['count'].min()

In [ ]:
# Aplicar log na variavel de resposta
df['count'] = np.log(df['count'])

## Forma mais elegante de fazer a transformação dos dados
## pois não perdemos os valores originais
## df['count_log'] = np.log(df['count'])

In [ ]:
# Verificando novamente a distribuição da variável count
df['count'].plot.hist(bins=50)

In [ ]:
# Visualizando os dados
df.head().T

In [ ]:
# Juntando os dataframes
df = df.append(test)

In [ ]:
# Convertendo a coluna datetime
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
# Verificando se funcionou
df.info()

# **Feature Engineering**

In [ ]:
# Criando novas colunas com a data e hora
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] =  df['datetime'].dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

In [ ]:
# Verificando os dados
df.info()

# **Separando os dataframes**

In [ ]:
# Primeiro o dataframe de  teste
test = df[df['count'].isnull()]

In [ ]:
# Depois o dataframe de  teste
df = df[~df['count'].isnull()]

In [ ]:
# Verificando tamanhos
df.shape, test.shape

# **Dividindo o dataframe de treino**

In [ ]:
# Importando o metodo do scikitlearn para divisao
from sklearn.model_selection import train_test_split

In [ ]:
# Dividir a base de treino
train, valid = train_test_split(df, random_state=42)

In [ ]:
# verificando tamanhos
train.shape, valid.shape

# **Selecionando as colunas que iremos usar como entrada**

In [ ]:
# Lista das colunas nao usadas
removed_cols = ['casual', 'registered', 'count', 'datetime']

# Criar a lista das colunas de entrada
feats = [c for c in train.columns if c not in removed_cols]

In [ ]:
# Verificando a lista de colunas
feats

# **Usando o modelo Random Forest**

In [ ]:
# Importando o modelo
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Instanciar o modelo
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['count'])

In [ ]:
# Fazendo previsões em cima dos dados de validação
preds = rf.predict(valid[feats])

In [ ]:
# Verificando as previsoes
preds

In [ ]:
# Verificando o real
valid['count'].head(3)

In [ ]:
# Verificando os dados reais do final do dataframe
valid['count'].tail(3)

# **Verificando o resultado do modelo com relação a métrica**

In [ ]:
# Importando a metrica
from sklearn.metrics import mean_squared_log_error

In [ ]:
# Aplicando a metrica
mean_squared_log_error(np.exp(valid['count']), np.exp(preds))**(1/2)

In [ ]:
# Vamos prever com base nos dados de treino
# Como o modelo se comporta prevendo em cima de dados conhecidos?
train_preds = rf.predict(train[feats])

mean_squared_log_error(np.exp(train['count']), np.exp(train_preds))**(1/2)

# **Gerando as previsões para envio ao Kaggle**

In [ ]:
# Gerando as previsões para envio ao Kaggle
test['count'] = np.exp(rf.predict(test[feats]))

In [ ]:
# Verificando os dados
test.head(10)

In [ ]:
# Verificando os dados a serem enviados
test[['datetime', 'count']].head()

In [ ]:
# Gerando o arquivo para submeter ao kaggle
test[['datetime', 'count']].to_csv('rf_novo1.csv', index=False)

# Aumentando a floresta -> 200 arvores aleatórias

In [ ]:
# Instanciando um novo modelo
rf2 = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=200,
                           min_samples_leaf=4)

# Treinando o modelo
rf2.fit(train[feats], train['count'])

# Fazendo as previsões na base de validação
preds2 = rf2.predict(valid[feats])

# Aplicando a metrica
mean_squared_log_error(np.exp(valid['count']), np.exp(preds2))**(1/2)

In [ ]:
# Gerando novamente as previsões para envio ao Kaggle
test['count'] = np.exp(rf2.predict(test[feats]))

# Gerando o arquivo
test[['datetime', 'count']].to_csv('rf_novo2.csv', index=False)

# Vamos adicionar colunas para obter a média da temperatura, da sensação térmica, da umidade e da velocidade do vento das horas anteriores

In [ ]:
# Explicando rolling
df[['datetime', 'temp']].head(10)

In [ ]:
# Juntando os dataframes
df = df.append(test)

In [ ]:
# Ordenando o dataframe
df.sort_values('datetime', inplace=True)

In [ ]:
# Criando a coluna rolling_temp
df['rolling_temp'] = df['temp'].rolling(3, min_periods=1).mean()

In [ ]:
# Verificando o resultado
df[['temp', 'rolling_temp']].head(10)

In [ ]:
# Criando a coluna rolling_atemp
df['rolling_atemp'] = df['atemp'].rolling(3, min_periods=1).mean()

In [ ]:
# Criando a coluna rolling_humidity
df['rolling_humidity'] = df['humidity'].rolling(3, min_periods=1).mean()

In [ ]:
# Criando a coluna rolling_windspeed
df['rolling_windspeed'] = df['windspeed'].rolling(3, min_periods=1).mean()

In [ ]:
# Visualizando os dados
df.head(10).T

In [ ]:
# Separando os dataframes

# Primeiro o teste
test = df[df['casual'].isnull()]

# Depois o treino
df = df[~df['casual'].isnull()]


In [ ]:
# Separando em treino e validação
train, valid = train_test_split(df, random_state=42)

In [ ]:
# Lista das colunas nao usadas
removed_cols = ['casual', 'registered', 'count', 'datetime']

# Criar a lista das colunas de entrada
feats = [c for c in train.columns if c not in removed_cols]

In [ ]:
# Instanciando um novo modelo
rf3 = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=200,
                           min_samples_leaf=4)

# Treinando o modelo
rf3.fit(train[feats], train['count'])

# Fazendo as previsões na base de validação
preds3 = rf3.predict(valid[feats])

# Aplicando a metrica
mean_squared_log_error(np.exp(valid['count']), np.exp(preds3))**(1/2)

In [ ]:
# Gerando novamente as previsões para envio ao Kaggle
test['count'] = np.exp(rf3.predict(test[feats]))

# Gerando o arquivo
test[['datetime', 'count']].to_csv('rf_novo3.csv', index=False)